In [25]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import time

In [26]:
class tutorial_model(nn.Module):
    def __init__(self):
        """ Initialize all layers of model """
        # TODO 
        super(tutorial_model, self).__init__()
        
        self.convolution1 = nn.Conv2d(in_channels = 3, out_channels = 10, kernel_size=5)
        self.convolution2 = nn.Conv2d(in_channels = 10, out_channels=20, kernel_size = 5)
        
        self.conv_dropout = nn.Dropout()
        
        self.fc1 = nn.Linear(in_features=320, out_features= 100)
        self.fc2 = nn.Linear(in_features=100, out_features=30)
        self.fc3 = nn.Linear(in_features=30, out_features=10)
        
        self.pool = nn.MaxPool2d(2,2)
        
        
    def forward(self, x):
        """ Chain all layers together """
        # TODO
        x= self.pool(F.relu(self.convolution1(x)))
        x= self.conv_dropout(x)
        x= self.pool(F.relu(self.convolution2(x)))
        
        x= x.view(-1,320)
        
        x= self.fc1(x)
        x= self.fc2(x)
        y= self.fc3(x)
        return y

In [27]:
# Number of epochs for training
num_epochs = 3     

# Batch Size for training/testing
batch_size = 64  

# Learning Rate for optimizer
learning_rate = 0.003

# Dimensions of CIFAR10
dim = 28   

In [28]:
# Transformation for training data
transform_train = torchvision.transforms.Compose([
    transforms.RandomResizedCrop(dim, scale=(0.7, 1.0), ratio=(1.0,1.0)),
    transforms.ColorJitter(
            brightness=0.1,
            contrast=0.1,
            saturation=0.1,
            hue=0.1),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),                            # Convert grayscale image to pytorch tensor
    transforms.Normalize((0.5,), (0.5,)),             # Normalize grayscale data
])

# Transformation for training data
transform_test = transforms.Compose([
    transforms.CenterCrop(dim),
    transforms.ToTensor(),                            # Convert grayscale image to pytorch tensor
    transforms.Normalize((0.5,), (0.5,)),             # Normalize grayscale data
])

In [29]:
# Download training data
trainset = torchvision.datasets.CIFAR10(root='./files', train=True, download=True, 
                                      transform=transform_train)

# Initialize dataloader for training data
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, 
                                           num_workers=8)

# Download testing Data
testset = torchvision.datasets.CIFAR10(root='./files', train=False, download=False, 
                                     transform=transform_test)

# Initialize dataloader for testing data
test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, 
                                          num_workers=8)

Files already downloaded and verified


In [22]:
# Initialize previously defined model
model = tutorial_model()                                               

# Definie loss function (Cross Entropy Loss)
criterion = nn.CrossEntropyLoss()                                      

# Initialize Optimizer (ADAM)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)     

# Set model to training (updating weights)
model.train();         

In [23]:
# Store time to calculate train time
start_time = time.time()

# Store loss and accuracy data
loss = []
accuracy = []

# Train the model
# Loop for number of epochs
for epoch in range(num_epochs):
    # Loop through data in batch sized increments
    for batch_idx, (X_train_batch, Y_train_batch) in enumerate(train_loader):
        # If trained on all data in epoch, move onto next epoch
        if(Y_train_batch.shape[0]<batch_size):
            continue

        # Forward pass through network
        output = model(X_train_batch)                           
        # Calculate loss of predictions
        curr_loss = criterion(output, Y_train_batch)            
        # Store loss
        loss.append(curr_loss.item())                           

        # Clear last calculation
        optimizer.zero_grad()                                   
        # Calculate gradient based on loss
        curr_loss.backward()                                    
        # Update model weights
        optimizer.step()                                        

        # Extract model predictions
        _, predicted = torch.max(output.data, 1) 
        # Calculate number of correct predictions
        correct = (predicted == Y_train_batch).sum().item()     
        # Calculate/store accuracy
        accuracy.append(correct/Y_train_batch.size(0))          
        
        # Intermitently print statistics
        if batch_idx % 100 == 0:
            print('Epoch: ' + str(epoch+1) + '/' + str(num_epochs) + ', Step: ' 
                  + str(batch_idx+1) + '/' + str(len(train_loader)) + ', Loss: ' 
                  + str(curr_loss.item()) + ', Accuracy: ' 
                  + str(correct/Y_train_batch.size(0)*100) + '%')

# Store time to calculate train time
end_time = time.time()

# Print train time
print('Run Time: ' + str(end_time - start_time))

Epoch: 1/3, Step: 1/782, Loss: 2.2883670330047607, Accuracy: 21.875%
Epoch: 1/3, Step: 101/782, Loss: 1.9429645538330078, Accuracy: 21.875%
Epoch: 1/3, Step: 201/782, Loss: 1.9058464765548706, Accuracy: 28.125%
Epoch: 1/3, Step: 301/782, Loss: 1.7356585264205933, Accuracy: 39.0625%
Epoch: 1/3, Step: 401/782, Loss: 1.6238470077514648, Accuracy: 35.9375%
Epoch: 1/3, Step: 501/782, Loss: 1.5989580154418945, Accuracy: 37.5%
Epoch: 1/3, Step: 601/782, Loss: 1.6437140703201294, Accuracy: 37.5%
Epoch: 1/3, Step: 701/782, Loss: 1.719868779182434, Accuracy: 31.25%
Epoch: 2/3, Step: 1/782, Loss: 1.7348352670669556, Accuracy: 42.1875%
Epoch: 2/3, Step: 101/782, Loss: 1.8617303371429443, Accuracy: 26.5625%
Epoch: 2/3, Step: 201/782, Loss: 1.5825626850128174, Accuracy: 37.5%
Epoch: 2/3, Step: 301/782, Loss: 1.5830882787704468, Accuracy: 45.3125%
Epoch: 2/3, Step: 401/782, Loss: 1.6057389974594116, Accuracy: 35.9375%
Epoch: 2/3, Step: 501/782, Loss: 1.5774242877960205, Accuracy: 46.875%
Epoch: 2/3, 

In [24]:
# Test the model
# Set model to testing (constant weights)
model.eval()

with torch.no_grad():
    # Store number of correct/total samples in test data
    correct = 0
    total = 0
    
    # Loop through test data
    for X_test_batch, Y_test_batch in test_loader:
        # Forward pass through network
        output = model(X_test_batch)  
        
        # Extract prediction
        _, predicted = torch.max(output.data, 1)    
        
        # Update total number of sample
        total += Y_test_batch.size(0)  
        
        # Update number of correct predictions
        correct += (predicted == Y_test_batch).sum().item()     

print('Test Accuracy: ' + str((correct/total) * 100) + '%')

Test Accuracy: 46.43%
